In [1]:
from utils import compute_f1, load_data
from text_train.imports import *
from text_train.data import TextDS, text_to_device
from arcface import ArcMarginProduct
from text_train.model import EMBBert
from text_train.train import *
np.random.seed(1337)
device = torch.device('cuda')

In [2]:
from imports import *

## Tests lr 

### mean agg

In [3]:
df, train_df, val_df, train_labels, val_labels = load_data(train_perc=0.3)

In [4]:
bs = 64
max_length=40
agg = 'mean_dumb'
language_model = 'bert-base-multilingual-uncased'
tokenizer = BertTokenizer.from_pretrained(language_model)

tr_ds = TextDS(train_df, tokenizer, max_length=40)
val_ds = TextDS(val_df, tokenizer, max_length=40)
full_ds = TextDS(df, tokenizer, max_length=40)

torch.save(tr_ds, 'data/text_models/datasets/tr_ds_0.3_{}.pth'.format(max_length))
torch.save(val_ds, 'data/text_models/datasets/val_ds_0.3_{}.pth'.format(max_length))
torch.save(full_ds, 'data/text_models/datasets/full_ds_0.3_{}.pth'.format(max_length))

In [5]:
tr_ds = torch.load('data/text_models/datasets/tr_ds_0.3_{}.pth'.format(max_length))
val_ds = torch.load('data/text_models/datasets/val_ds_0.3_{}.pth'.format(max_length))
full_ds = torch.load('data/text_models/datasets/full_ds_0.3_{}.pth'.format(max_length))

In [6]:
tr_test_dl = DataLoader(tr_ds, batch_size = bs, shuffle = False, pin_memory = True)
tr_dl = DataLoader(tr_ds, batch_size = bs, shuffle = True, pin_memory = True)
val_dl = DataLoader(val_ds, batch_size = bs, shuffle = False, pin_memory = True)
full_dl = DataLoader(full_ds, batch_size = bs, shuffle = False, pin_memory = True)

In [7]:
# model
model = EMBBert(language_model, agg=agg).to(device)

centers = compute_centers(tr_test_dl, model, train_df)
torch.save(centers, 'data/text_models/centers_0.3_{}_{}.pth'.format(agg, max_length))

In [8]:
centers = torch.load('data/text_models/centers_0.3_{}_{}.pth'.format(agg, max_length))

In [9]:
metric_fc = ArcMarginProduct(512, train_df['label_group'].nunique(), s=30, m=0.5,
                             easy_margin=False, centers=centers, device=device).to(device)

Using center as wieghts


In [10]:
n_epochs, lf, params, optimizer, sched = get_hparams(tr_dl, model, metric_fc, lr=5e-4, n_epochs=10, sched=True)

In [11]:
loss_hist = []
best_thr_score={'val': {'thr': None, 'f1': None}, 'train': {'thr': None, 'f1': None}}
thr_score_hist=[]
ep_start = 0
save_path = 'data/text_models/test_3n_epochs'

In [12]:
best_thr_score, thr_score_hist, losses = train(model, optimizer, lf, sched, metric_fc, tr_dl, val_dl,
                                               n_epochs, train_df, val_df, 
                                               save_path=save_path, 
                                               prev_best_info=best_thr_score, info_history=thr_score_hist,
                                               ep_start=ep_start, device=device)
loss_hist.append(losses)
ep_start += n_epochs

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

### cls agg

In [3]:
from utils import compute_f1, load_data
from text_train.imports import *
from text_train.data import TextDS, text_to_device
from arcface import ArcMarginProduct
from text_train.model import EMBBert
from text_train.train import *
np.random.seed(1337)
device = torch.device('cuda')

In [4]:
from imports import *

In [5]:
df, train_df, val_df, train_labels, val_labels = load_data(train_perc=0.3)

In [6]:
bs = 64
language_model = 'bert-base-multilingual-uncased'
tokenizer = BertTokenizer.from_pretrained(language_model)

tr_ds = TextDS(train_df, tokenizer)
tr_test_dl = DataLoader(tr_ds, batch_size = bs, shuffle = False, pin_memory = True)

tr_ds = TextDS(train_df, tokenizer)
tr_dl = DataLoader(tr_ds, batch_size = bs, shuffle = True, pin_memory = True)

val_ds = TextDS(val_df, tokenizer)
val_dl = DataLoader(val_ds, batch_size = bs, shuffle = False, pin_memory = True)

full_ds = TextDS(df, tokenizer)
full_dl = DataLoader(full_ds, batch_size = bs, shuffle = False, pin_memory = True)

torch.save(tr_test_dl, 'data/text_models/tr_test_dl_0.3.pth')
torch.save(tr_dl, 'data/text_models/tr_dl_0.3.pth')
torch.save(val_dl, 'data/text_models/val_dl_0.3.pth')
torch.save(full_dl, 'data/text_models/full_dl_0.3.pth')

In [7]:
tr_dl = torch.load('data/text_models/tr_dl_0.3.pth')
val_dl = torch.load('data/text_models/val_dl_0.3.pth')
tr_test_dl = torch.load('data/text_models/tr_test_dl_0.3.pth')
full_dl = torch.load('data/text_models/full_dl_0.3.pth')

In [8]:
# model
model = EMBBert(language_model, agg='cls').to(device)

centers = compute_centers(tr_test_dl, model, train_df)
torch.save(centers, 'data/text_models/centers_0.3.pth')

In [9]:
centers = torch.load('data/text_models/centers_0.3.pth')

In [10]:
metric_fc = ArcMarginProduct(512, train_df['label_group'].nunique(), s=30, m=0.5,
                             easy_margin=False, centers=centers, device=device).to(device)

Using center as wieghts


In [11]:
n_epochs, lf, params, optimizer, sched = get_hparams(tr_dl, model, metric_fc, lr=5e-4, n_epochs=5, sched=None)

In [12]:
loss_hist = []
best_thr_score={'val': {'thr': None, 'f1': None}, 'train': {'thr': None, 'f1': None}}
thr_score_hist=[]
ep_start = 0
save_path = 'data/text_models/test_3n_epochs'

In [13]:
best_thr_score, thr_score_hist, losses = train(model, optimizer, lf, None, metric_fc, tr_dl, val_dl,
                                               n_epochs, train_df, val_df, 
                                               save_path=save_path, 
                                               prev_best_info=best_thr_score, info_history=thr_score_hist,
                                               ep_start=ep_start, device=device)
loss_hist.append(losses)
ep_start += n_epochs

Checkpoint : saved model to data/text_models/test_3n_epochs_ep_0.pth


Saved best model ep 0 with f score : 0.0003401704083750586
Ep 0: Train loss 15.8120 | Val f score 0.0003 with thresh 0.20, train f score 0.4530 with thresh 0.20


KeyboardInterrupt: 

## Translate

In [3]:
df, train_df, val_df, train_labels, val_labels = load_data(train_perc=0.3)

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("../input/id-en-converter")
txt_model = AutoModelForSeq2SeqLM.from_pretrained("../input/id-en-converter")

In [7]:
#tokenizer = AutoTokenizer.from_pretrained("../id-en-converter")
txt_model = AutoModelForSeq2SeqLM.from_pretrained("../input/id-en-converter")

404 Client Error: Not Found for url: https://huggingface.co/input/id-en-converter/resolve/main/config.json


OSError: Can't load config for '../input/id-en-converter'. Make sure that:

- '../input/id-en-converter' is a correct model identifier listed on 'https://huggingface.co/models'

- or '../input/id-en-converter' is the correct path to a directory containing a config.json file

